In [1]:
# import libraries
## ffmpeg library enables the use of FFmpeg from within Python code
import ffmpeg
import math 
import os

In [2]:
def videoDataExtractor(f):

    metadata = ffmpeg.probe(f)

    # Extract the video and audio streams from the probe results
    video_stream = next((stream for stream in metadata['streams'] if stream['codec_type'] == 'video'), None)
    audio_stream = next((stream for stream in metadata['streams'] if stream['codec_type'] == 'audio'), None)

    # Extract the required information from the video stream
    video_format = f.split(".")[1]
    video_codec = video_stream['codec_name']
    frame_rate_data = video_stream['avg_frame_rate'].split("/")
    frame_rate = round(int(frame_rate_data[0])/int(frame_rate_data[1]))
    if 'display_aspect_ratio' in video_stream:
        aspect_ratio = video_stream['display_aspect_ratio']
    else:
        width, height = video_stream['width'], video_stream['height']
        aspect_ratio = str(int(width / math.gcd(width, height))) + ':'+ str(int(height / (math.gcd(width, height))))
    resolution = f"{video_stream['width']}x{video_stream['height']}"
    video_bitrate = int(video_stream['bit_rate'])/1000/1000

    # Extract the required information from the audio stream
    audio_codec = audio_stream['codec_name']
    audio_bitrate = int(audio_stream['bit_rate'])/1000
    audio_channels = audio_stream['channels']

    # Print the extracted information
    print(f"Video format: {video_format}")
    print(f"Video codec: {video_codec}")
    print(f"Audio codec: {audio_codec}")
    print(f"Frame rate: {frame_rate} FPS")
    print(f"Aspect ratio: {aspect_ratio}")
    print(f"Resolution: {resolution}")
    print(f"Video bit rate: {video_bitrate:.2f} Mb/s")
    print(f"Audio bit rate: {audio_bitrate:.2f} kb/s")
    print(f"Audio channels: {audio_channels}")
    
    return video_format,video_codec,audio_codec,frame_rate,aspect_ratio,resolution,video_bitrate,audio_bitrate,audio_channels

In [3]:
def videoChecker(fileName,videoFormat,videoCodec,audioCodec,frameRate,aspectRatio,res,videoBitRate,audioBitRate,audioChannel):
    # create storage
    clearedList = []
    reportTxt = ""
    
    # write report
    with open('films_report.txt', 'a') as f:
        #f.write('readme')
        
        # Check the information and report what is wrong
        ## video format must be mp4
        print(f"Video format: {video_format}")
        if video_format == "mp4":
            print("Format is ok\n")
            # no append needed for list, since format/container change will be included in output file
        else:
            print("Format is wrong, needs to be mp4\n")
            # append to string
            reportTxt = reportTxt+f"Wrong format, {video_format} is not mp4\n"
        
        ## video codec must be h.264
        print(f"Video codec: {video_codec}")
        if video_codec == "h264":
            print("Video codec is ok\n")
            # append to clearedList
            clearedList.append("vcodec")
        else:
            print("Video codec is wrong, needs to be h.264\n")
            # append to string
            reportTxt = reportTxt+f"Wrong video codec, {video_codec} is not h.264\n"
            
        print(f"Audio codec: {audio_codec}")
        if audio_codec == "aac":
            print("Audio codec is ok\n")
            # append to clearedList
            clearedList.append("acodec")
        else:
            print("Audio codec is wrong, needs to be aac\n")
            # append to string
            reportTxt = reportTxt+f"Wrong audio codec, {audio_codec} is not aac\n"
            
        print(f"Frame rate: {frame_rate} FPS")
        if frame_rate == 25:
            print("Format is ok\n")
            # append to clearedList
            clearedList.append("framerate")
        else:
            print("Frame rate is wrong, needs to be 25 FPS\n")
            # append to string
            reportTxt = reportTxt+f"Wrong frame rate, {frame_rate} is not 25 FPS\n"
            
        print(f"Aspect ratio: {aspect_ratio}")
        if aspect_ratio == "16:9":
            print("Aspect ratio is ok\n")
            # append to clearedList
            clearedList.append("aspectratio")
        else:
            print("Aspect ratio is wrong, needs to be 16:9\n")
            # append to string
            reportTxt = reportTxt+f"Wrong aspect ratio, {aspect_ratio} is not 16:9\n"
            
        print(f"Resolution: {resolution}")
        if resolution == "640x360":
            print("Resolution is ok\n")
            # append to clearedList
            clearedList.append("resolution")
        else:
            print("Resolution is wrong, needs to be 640x360\n")
            # append to string
            reportTxt = reportTxt+f"Wrong resolution, {resolution} is not 640x360\n"
        
        print(f"Video bit rate: {video_bitrate:.2f} Mb/s")
        if video_bitrate >= 2 and video_bitrate <= 5:
            print("Video bit rate is ok\n")
            # append to clearedList
            clearedList.append("vbitrate")
        else:
            print("Video bit rate is wrong, needs to be 2-5 Mb/s \n")
            # append to string
            reportTxt = reportTxt+f"Wrong video bit rate, {video_bitrate:.2f} is not 2-5 Mb/s \n"
            
        print(f"Audio bit rate: {audio_bitrate:.2f} kb/s")
        if audio_bitrate <= 256:
            print("Audio bit rate is ok\n")
            # append to clearedList
            clearedList.append("abitrate")
        else:
            print("Audio bit rate is wrong, needs to be 256 kb/s and below\n")
            # append to string
            reportTxt = reportTxt+f"Wrong audio bit rate, {audio_bitrate:.2f} is not 256 kb/s and below\n"
            
        print(f"Audio channels: {audio_channels}")
        if audio_channels == 2:
            print("Audio channels is ok\n")
            # append to clearedList
            clearedList.append("audiochannel")
        else:
            print("Audio channels is wrong, needs to be stereo\n")
            # append to string
            reportTxt = reportTxt+f"Wrong audio channels, {audio_channels} is not stereo\n"
        
        reportTxt = fileName+" does not respect the specified format\n"+ reportTxt + "\n"
        print(reportTxt)
        f.write(reportTxt)
        return clearedList

In [4]:
def videoEditor(inputFile, clearedList):
    # Input file
    input_stream = ffmpeg.input(inputFile)
    
    # Output file
    output_file = inputFile.split(".")[0]+"_formatOK.mp4"
    
    # Set video parameters
    video_codec = 'libx264'
    video_bitrate = '5M'
    frame_rate = 25
    aspect_ratio = '16:9'
    resolution = '640x360'

    # Set audio parameters
    audio_codec = 'aac'
    audio_bitrate = '256k'
    audio_channels = 2
    
    # retrieve params
    output_args = {'vcodec': video_codec, 'b': video_bitrate, 'framerate': frame_rate, 'aspect': aspect_ratio, 's': resolution, 'acodec': audio_codec, 'ab': audio_bitrate, 'ac': audio_channels}
    
    if len(clearedList)>0:
        #if argument is inside clearedList, remove the argument(since no need to change)
        for argument in clearedList:
            if argument == "vcodec":
                output_args.pop('vcodec')

            elif argument == "vbitrate":
                output_args.pop('b')
                
            elif argument == "framerate":
                output_args.pop('framerate')

            elif argument == "aspectratio":
                output_args.pop('aspect')

            elif argument == "resolution":
                output_args.pop('s')

            elif argument == "acodec":
                output_args.pop('acodec')

            elif argument == "abitrate":
                output_args.pop('ab')

            elif argument == "audiochannel":
                output_args.pop('ac')

    output_stream = input_stream.output(output_file,**output_args)
    # Run ffmpeg command
    ffmpeg.run(output_stream)

In [5]:
# create empty films_report.txt file
with open('films_report.txt', 'w') as f:
    f.write("Problematic Films\n")
    f.close()

# iterate over files in directory "films" and run functions
directory = 'films'

for filename in os.listdir(directory):
    vidfile = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(vidfile):
        video_format,video_codec,audio_codec,frame_rate,aspect_ratio,resolution,video_bitrate,audio_bitrate,audio_channels = videoDataExtractor(vidfile)
        clearedList = videoChecker(filename,
                                   video_format,video_codec,audio_codec,frame_rate,aspect_ratio,
                                   resolution,video_bitrate,audio_bitrate,audio_channels)
        videoEditor(vidfile, clearedList)
        f.close()

Video format: mp4
Video codec: h264
Audio codec: aac
Frame rate: 30 FPS
Aspect ratio: 314:177
Resolution: 628x354
Video bit rate: 2.99 Mb/s
Audio bit rate: 317.10 kb/s
Audio channels: 2
Video format: mp4
Format is ok

Video codec: h264
Video codec is ok

Audio codec: aac
Audio codec is ok

Frame rate: 30 FPS
Frame rate is wrong, needs to be 25 FPS

Aspect ratio: 314:177
Aspect ratio is wrong, needs to be 16:9

Resolution: 628x354
Resolution is wrong, needs to be 640x360

Video bit rate: 2.99 Mb/s
Video bit rate is ok

Audio bit rate: 317.10 kb/s
Audio bit rate is wrong, needs to be 256 kb/s and below

Audio channels: 2
Audio channels is ok

Cosmos_War_of_the_Planets.mp4 does not respect the specified format
Wrong frame rate, 30 is not 25 FPS
Wrong aspect ratio, 314:177 is not 16:9
Wrong resolution, 628x354 is not 640x360
Wrong audio bit rate, 317.10 is not 256 kb/s and below


Video format: mov
Video codec: prores
Audio codec: pcm_s16le
Frame rate: 24 FPS
Aspect ratio: 16:9
Resolution: